# 서대문구 (1093)
- 작성자 : 황다은
- 작성일자 : 2020년 10월 3일
- 감염경로가 적혀있는 경우도 있으나 비고란에 조치 및 선별진료소, 해외입국자 여부 등등이 섞여있어 전처리 불가.

In [111]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

dat = dt.datetime.today().date() # 오늘 날짜

### 1. Raw File 불러오기

In [112]:
df = pd.read_csv("200922one_shot.csv")
soup = BeautifulSoup(df['html'][1], 'html.parser')

### 2. 간단한 전처리 하기

In [113]:
rows = soup.select('.viewCon')[0].select(' .table_st')
df = pd.read_html(str(rows))[0]

start = int(df.iloc[0,0][1:])

end = int(df.iloc[len(df)-1,0][1:])


### 3. 저장할 DataFrame 미리 만들기

In [119]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt','age','sex']
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']
patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [120]:
for i in tqdm(range(int(end),int(start)+1)):

    num = '1093' + '0' * (5- len(str(i))) + str(i)
    sub = df['연번']=='#'+str(i)
    subdb = df[sub]
    cnd = subdb.iloc[0,1].split('  ')[0]
    rsd = subdb.iloc[0,1].split('  ')[1]
    syy = ''; syd = ''; pth = ''; atn = ''; dch = ''; adt = ''; age = ''; sex = ''
    new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt, age, sex)], columns = pat_col)
    patient = pd.concat([patient, new_pat])

    new_route = subdb.iloc[0:len(subdb)-1,:].copy()
    new_route['exd'] = subdb['노출일시'][0:len(subdb)-1].str.replace("\(마스크 착용\)","")
    new_route['msk'] = subdb['노출일시'][0:len(subdb)-1].str.contains('마스크 착용')
    new_route['num'] = num
    new_route['ord'] = range(1, len(new_route)+1)
    new_route['dat'] = dat
    new_route['mob'] = ''
    new_route = new_route.drop(new_route.columns[[0, 1, 5]], axis=1)
    new_route.columns = ['rgt', 'frm', 'rgl', 'ste', 'exd', 'msk', 'num', 'ord', 'dat', 'mob']
    new_route = new_route.loc[:,rou_col]

    route = pd.concat([route, new_route])


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 31.72it/s]


In [121]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt,age,sex
0,109300104,2020-10-03,,,,9.7,연희동,,,,,
0,109300105,2020-10-03,,,,9.8.,충현동,,,,,
0,109300106,2020-10-03,,,,9.8.,홍제3동,,,,,
0,109300107,2020-10-03,,,,9.9.,연희동,,,,,
0,109300108,2020-10-03,,,,9.9.,남가좌1동,,,,,
0,109300109,2020-10-03,,,,9.10.,홍은1동,,,,,
0,109300110,2020-10-03,,,,9.10,남가좌2동,,,,,
0,109300111,2020-10-03,,,,9.10.,연희동,,,,,
0,109300112,2020-10-03,,,,9.11.,홍은1동,,,,,
0,109300113,2020-10-03,,,,9.11.,연희동,,,,,


In [122]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
57,109300104,1,2020-10-03,연희동,음식점,**칼국수,2020.09.03.(목) 12:15~13:30,True,소독완료,
58,109300104,2,2020-10-03,신촌동,음식점,**덮밥집,2020.09.04.(금) 12:25~12:55,True,소독완료,
59,109300104,3,2020-10-03,신촌동,음식점,**카페,2020.09.04.(금) 13:00~13:40,True,소독완료,
60,109300104,4,2020-10-03,창천동,상점,**신발점,2020.09.04.(금) 17:00~18:05,True,소독완료,
61,109300104,5,2020-10-03,연희동,보건소,서대문구보건소,2020.09.06.(일) 12:00~13:00,True,-,
52,109300106,1,2020-10-03,홍은동,마트,**마트,2020.09.05.(토) 17:45~17:54,True,소독완료,
53,109300106,2,2020-10-03,홍제동,의료기관,*****의원,2020.09.07.(월) 11:35~11:44,True,소독완료,
54,109300106,3,2020-10-03,홍제동,약국,**약국,2020.09.07.(월) 11:45~11:47,True,소독완료,
46,109300107,1,2020-10-03,남가좌동,음식점,**음식점,2020.09.03.(목) 19:10~19:30,True,소독완료,
47,109300107,2,2020-10-03,남가좌동,음식점,**빵집,2020.09.03.(목) 19:37~19:50,True,소독완료,
